In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
import numpy as np
PATH = Path("data/matify/image")
os.makedirs(PATH,exist_ok=True)
torch.cuda.set_device(1)

In [3]:
from shutil import copy2
PATH_SLT = Path("data/matify/image_slt")
PATH_SLT_TRN = PATH_SLT/"train"
PATH_SLT_VAL = PATH_SLT/"val"
r_test = 0.2
PATH_SLT.mkdir(exist_ok=True)
PATH_SLT_TRN.mkdir(exist_ok=True)
PATH_SLT_VAL.mkdir(exist_ok=True)
for cls in PATH.iterdir():
    cls_name = cls.name
    TRN = PATH_SLT_TRN/cls_name
    TRN.mkdir(exist_ok=True)
    VAL = PATH_SLT_VAL/cls_name
    VAL.mkdir(exist_ok=True)
    
    imgs = list(cls.iterdir())
    idx = np.random.permutation(range(len(imgs)))
    slt = int(len(imgs)*0.2)
    val_ids = idx[:slt]; trn_ids = idx[slt:]
    
    for trn_idx in trn_ids:
        img_name = imgs[trn_idx].name
        (TRN/img_name).symlink_to(imgs[trn_idx].absolute())
    for val_idx in val_ids:
        img_name = imgs[val_idx].name
        (VAL/img_name).symlink_to(imgs[val_idx].absolute())

In [4]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "data/dogscats/"
sz=299
arch=resnext50
bs=28
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH_SLT, val_name='val', tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

100%|██████████| 32/32 [00:05<00:00,  6.10it/s]


In [5]:
learn.fit(1e-2, 1)
learn.precompute=False

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      2.512052   1.828541   0.538721  



In [6]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.983989   1.674356   0.566779  
    1      1.782623   1.560642   0.612795                   



[array([1.56064]), 0.6127946092491064]

In [7]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [8]:
learn.fit(lr, 10, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.704599   1.280638   0.673401  
    1      1.193648   1.052215   0.717172                   
    2      0.927924   0.947193   0.74523                     
    3      0.700896   0.880788   0.776655                    
    4      0.539755   0.813714   0.791246                    
    5      0.431486   0.821294   0.777778                    
    6      0.379475   0.77573    0.794613                    
    7      0.319612   0.768474   0.804714                    
    8      0.275784   0.753508   0.801347                    
    9      0.224121   0.755023   0.811448                    



[array([0.75502]), 0.8114478155284885]

In [10]:
learn.fit(lr, 10, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.078264   0.767854   0.818182  
    1      0.090319   0.788411   0.819304                     
    2      0.072055   0.786492   0.81257                      
    3      0.08997    0.807723   0.813692                     
    4      0.073332   0.819845   0.806958                     
    5      0.060946   0.809367   0.813692                     
    6      0.058102   0.795132   0.81257                      
    7      0.07119    0.801705   0.822671                     
    8      0.059138   0.803044   0.815937                     
    9      0.061958   0.814675   0.811448                     



[array([0.81468]), 0.8114478117822932]